# ReAct

This agent uses ReAct to reason about what to do.

Logic "deconstructed" from LlamaIndex

In [1]:
# !pip install openai agents_deconstructed

In [10]:
from langchain.agents import tool
from langchain.schema.agent import AgentFinish
from langchain.callbacks.manager import (
    trace_as_chain_group,
)
from langchain import hub
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import ChatOpenAI

In [3]:
from agents_deconstructed import react_chat
from agents_deconstructed.format_tools import format_tools_args

In [4]:
# Create a dummy tool
@tool
def run_code(query: str):
    """Input should be Python code. Use this to do math."""
    return eval(query)

In [19]:
# Create our prompt, either directly or from hub
prompt = ChatPromptTemplate.from_messages([
    ("system", react_chat.REACT_AGENT_INSTRUCTIONS),
    ("user", "{question}"),
    MessagesPlaceholder(variable_name="intermediate_steps"),
])
# Or call pull from hub!
#prompt = hub.pull("shoggoth13/react-chat-agent")

In [20]:
tools = [run_code]

In [21]:
prompt = prompt.partial(
    tool_desc=format_tools_args(tools),
    tool_names=", ".join([tool.name for tool in tools])
)

In [22]:
agent = prompt | ChatOpenAI(temperature=0) | react_chat.ReActOutputParser()

In [23]:
# Use this to group all traces together
with trace_as_chain_group("agent-run") as group_manager:
    # Keep track of steps taken
    steps = []
    # Get the first response from the agent
    action = agent.invoke({
        "question": "whats 2 + 2",
        "intermediate_steps": react_chat.format_steps(steps)
    }, config={"callbacks": group_manager})

    # While it's not finished, iterate
    while not isinstance(action, AgentFinish):
        # Select the tool to use
        tool = {
            "run_code": run_code,
        }[action.tool]
        # Run the tool
        observation = tool.run(action.tool_input, callbacks=group_manager)
        # Construct intermediate steps
        steps += [(action, observation)]
        # Log the steps
        print(steps)
        # Do the next iteration
        action = agent.invoke({
            "question": "whats 2 + 2",
            "intermediate_steps": react_chat.format_steps(steps)
        }, config={"callbacks": group_manager})

[(AgentAction(tool='run_code', tool_input={'query': '2 + 2'}, log='I can use the `run_code` tool to perform the addition.'), 4)]


In [24]:
action

AgentFinish(return_values={'output': '4'}, log='I can answer without using any more tools.')